In [ ]:
##### OpenAI Key 方案 #####
from dotenv import load_dotenv
load_dotenv(".env")
    
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding()

# test chat
response = llm.complete("香蕉的颜色是")
print(response)

In [ ]:
##### 本地模型加载方案 #####
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 加载本地的qwen2-7b，你本地的模型放在哪就直接改路径
llm = HuggingFaceLLM(
    tokenizer_name="G:\\models\\Qwen2-7B-Instruct",
    model_name="G:\\models\\Qwen2-7B-Instruct",
    device_map="auto",
    tokenizer_kwargs={"trust_remote_code": True},
    model_kwargs={"trust_remote_code": True},
)
# 调用本地bce-embedding-base_v1作为embedding模型
embed_args = {
    'model_name': 'hkunlp/instructor-base', 
    'max_length': 512, 
    'embed_batch_size': 32, 
    'device': 'cuda'
    }
embed_model = HuggingFaceEmbedding(**embed_args)

In [ ]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def add(x: int, y: int) -> int:
    """将两个数相加"""
    return x + y

# substraction function
def sub(x: int, y: int) -> int:
    """两个数字相减"""
    return x - y

# multiplication function
def mul(x: int, y: int) -> int:
    """两个数字相乘"""
    return x * y


# get user information
def get_user_info(name: str) -> str:
    """Get user information."""
    data = {
        "雄哥": {
            "age": 18,
            "location": "广东"
        },
        "小胖": {
            "age": 60,
            "location": "广东"
        }
    }
    return f'名字 {name}, 年龄 {data[name]["age"]} 来自 {data[name]["location"]}'

In [ ]:
from llama_index.core.tools import FunctionTool

addition_tool = FunctionTool.from_defaults(fn=add)
get_user_info_tool = FunctionTool.from_defaults(fn=get_user_info)
multiplication_tool = FunctionTool.from_defaults(fn=mul)
substraction_tool = FunctionTool.from_defaults(fn=sub)

tools = [addition_tool, get_user_info_tool, multiplication_tool, substraction_tool]

In [ ]:
# 这里试试上面定义的函数，大模型能否正常使用
response = llm.predict_and_call(
    tools, 
    "5乘以5等于多少？", 
    verbose=True
)
print(str(response))

In [ ]:
# 再试一下，确定能正常使用
response = llm.predict_and_call(
    tools, 
    "雄哥多少岁？", 
    verbose=True  # 把思考的过程也进行打印输出
)
print(str(response))

In [ ]:
from llama_index.core import SimpleDirectoryReader

# 加载PDF数据，这里雄哥以卖油翁为例，可以改为自己的任意数据放在datasets文件夹即可，或改为指定的绝对路径
documents = SimpleDirectoryReader(input_files=["./datasets/maiyouweng.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

# 把文本分块，chunk_size为1024，每一块的大小
splitter = SentenceSplitter(chunk_size=1024)
# 创建文档的节点
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
# 这里可以打印每一个块的信息！
node_metadata = nodes[0].get_content(metadata_mode=True)
print(node_metadata)

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# 创建数据摘要索引
summary_index = SummaryIndex(nodes)
# 创建矢量存储索引
vector_index = VectorStoreIndex(nodes)

In [ ]:
from llama_index.core.vector_stores import MetadataFilters

# Create vector search query engine
query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "康肃公陈尧咨善于射箭，曾经，他在家里场地射箭，然后发生了什么事？", 
)
print(str(response))

In [ ]:
print(len(response.source_nodes))
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text())
    print("=============Text=============")

In [ ]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_search_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """使用以下参数在索引中进行向量搜索：

    query (str): 这是你想要在索引中嵌入和搜索的文本字符串
    page_numbers (List[str]): 这个参数允许你将搜索限制到特定的页面。如果留空，搜索将包含索引中的所有页面。如果指定了页码，搜索将只包括那些页面
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

In [ ]:
vector_query_tool = FunctionTool.from_defaults(
    name="vector_search_tool",
    fn=vector_search_query
)

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool], 
    "第二页提到了什么内容？", 
    verbose=True
)

In [ ]:
# 确认系统在哪里检索的数据，返回数据源
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text())
    print("=============Text=============")

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "关于卖油翁的摘要生成"
    ),
)

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "第一页提到了什么内容？",
    verbose=True
)

In [ ]:
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text()[:10])
    print("=============Text=============")

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "给我这篇卖油翁课堂的摘要！",
    verbose=True
)

In [ ]:
for n in response.source_nodes:
    print(n.metadata)
    print("=============Text=============")
    print(n.get_text()[:10])
    print("=============Text=============")